# [LVV-T2216] Integrated Slew and Tracking Test - Data Analysis

This notebook is used to analyse the data colleted when running the [LVV-T2216 (1.0)] test case during System Spread Integration Tests on Level 3.  
   
**Requirements:**
 - You should have run this test case and record the time stamps on each step.
 
Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

**Make sure you run this notebook on TTS before running at the summit.**

Please, see the [README] file for the requirements to run this notebook.

[LVV-T2216]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2216
[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md

In [ ]:
test_case = "LVV-T2216"
test_exec = "LVV-EXXXX"

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np

from astropy.time import Time
from matplotlib import pyplot as plt

from lsst.sitcom import vandv

In [ ]:
client = vandv.efd.create_efd_client()

exec_info = vandv.ExecutionInfo()
print(exec_info)

## Slewing without LUT corrections



In [ ]:
# t_start = "2022-07-20 19:18:09.246021""
# t_end = "2022-07-20 19:22:07.383914"

In [ ]:
df = await vandv.efd.query_script_message_contains(
    client=client, 
    contains=[test_case, test_exec, "START", "Slewing without LUT"],
    upper_t=Time.now(),
    lower_t=Time("2022-06-01 17:33", scale="utc", format="iso"),
    num=1)

t_start = vandv.efd.time_pd_to_astropy(df.index[0].floor("1s"))
print(t_start, df.iloc[0]["message"])

In [ ]:
df = await vandv.efd.query_script_message_contains(
    client=client, 
    contains=[test_case, test_exec, "END", "Slewing without LUT"],
    upper_t=Time.now(),
    lower_t=Time("2022-06-01 17:33", scale="utc", format="iso"),
    num=1)

t_end = vandv.efd.time_pd_to_astropy(df.index[0].ceil("1s"))
print(t_end, df.iloc[0]["message"])

In [ ]:
print(f"Starting data analysis between\n  {t_start}\n    and\n  {t_end}")

### Analyse M1M3 Data

In [ ]:
fad = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    "*", 
    t_start,
    t_end)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    "*", 
    t_start,
    t_end)

fig, axs = plt.subplot_mosaic(
    mosaic="A",
    num="Slew Without Correction", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 4),
)

vandv.m1m3.timeline_zforces(axs["A"], fad, "zForce", elevation=el)

In [ ]:
fel = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedElevationForces", 
    "*", 
    t_start,
    t_end)

fba = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedBalanceForces", 
    "*",
    t_start,
    t_end)

fst = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedStaticForces",
    "*",
    t_start,
    t_end)

fao = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedActiveOpticForces",
    "*", 
    t_start,
    t_end)

series = [df.iloc[-1] for df in [fel, fba, fst, fad]]
labels = [
    "appliedElevationForces", 
    "appliedBalanceForces", 
    "appliedStaticForces",
    "forceActuatorData",
]

fig, axs = plt.subplot_mosaic(
    mosaic="AB\nAB\nCC\nCC",
    num="Slew Without Correction - Snapshot Forces", 
    constrained_layout=True,
    dpi=120,
    figsize=(9, 5),
)

vandv.m1m3.snapshot_xforces(axs["A"], series, labels=labels)
vandv.m1m3.snapshot_yforces(axs["B"], series, labels=labels)
vandv.m1m3.snapshot_zforces(axs["C"], series)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AB\nCD",
    num="Slew Without Correction - Snapshot zForces Overview", 
    constrained_layout=False,
    dpi=120,
    figsize=(10, 10),
)

vandv.m1m3.snapshot_zforces_overview(axs["A"], fad.iloc[-1], title="forceActuatorData")
vandv.m1m3.snapshot_zforces_overview(axs["B"], fel.iloc[-1], title="appliedElevationForces")
vandv.m1m3.snapshot_zforces_overview(axs["C"], fba.iloc[-1], title="appliedBalanceForces")
vandv.m1m3.snapshot_zforces_overview(axs["D"], fao.iloc[-1], title="appliedActiveOpticForces")

### Analyse M2

In [ ]:
axf = await client.select_time_series(
    "lsst.sal.MTM2.axialForce", 
    "*",
    t_start,
    t_end)

taf = await client.select_time_series(
    "lsst.sal.MTM2.tangentForce", 
    "*",
    t_start,
    t_end)

cof = await client.select_time_series(
    "lsst.sal.MTM2.command_applyForces",
    "*", 
    t_start,
    t_end)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="A\nB\nC",
    num="Slew Without Correction - M2", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 8),
)

_ = vandv.m2.timeline_axial_forces(axs["A"], axf, elevation=el)
_ = vandv.m2.timeline_axial_forces(axs["B"], axf, elevation=el, column="applied")
_ = vandv.m2.timeline_axial_forces(axs["C"], axf, elevation=el, column="lutGravity")

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="A\nB\nC\nD",
    num="Slew Without Correction - M2", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 8),
)

cols = [
    # "applied", 
    # "hardpointCorrection", 
    "lutGravity", 
    # "lutTemperature", 
    "measured"
]

_ = vandv.m2.timeline_axial_forces_per_act(axs["A"], axf, elevation=el, act="B1", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["B"], axf, elevation=el, act="B8", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["C"], axf, elevation=el, act="B16", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["D"], axf, elevation=el, act="B24", cols=cols)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="ABC",
    num="Slew Without Correction - M2 All Actuators", 
    constrained_layout=False,
    dpi=120,
    figsize=(12, 6),
)

_ = vandv.m2.snapshot_zforces_overview(axs["A"], axf.iloc[-1], ms=150, fs=6)
_ = vandv.m2.snapshot_zforces_overview(axs["B"], axf.iloc[-1], prefix="lutGravity", ms=150, fs=6)
_ = vandv.m2.snapshot_zforces_overview(axs["C"], axf.iloc[-1], prefix="applied", ms=150, fs=6)

## Slewing with LUT corrections

In [ ]:
t_start = "2022-07-20 19:28:29.149291"
t_end = "2022-07-20 19:32:28.344625"

In [ ]:
df = await vandv.efd.query_script_message_contains(
    client=client, 
    contains=[test_case, test_exec, "START", "Slewing with LUT"],
    upper_t=Time.now(),
    lower_t=Time("2022-06-01 17:33", scale="utc", format="iso"),
    num=1)

t_start = vandv.efd.time_pd_to_astropy(df.index[0].floor("1s"))
print(t_start, df.iloc[0]["message"])

In [ ]:
df = await vandv.efd.query_script_message_contains(
    client=client, 
    contains=[test_case, test_exec, "END", "Slewing with LUT"],
    upper_t=Time.now(),
    lower_t=Time("2022-06-01 17:33", scale="utc", format="iso"),
    num=1)

t_end = vandv.efd.time_pd_to_astropy(df.index[0].ceil("1s"))
print(t_end, df.iloc[0]["message"])

In [ ]:
print(f"Starting data analysis between\n  {t_start}\n    and\n  {t_end}")

### Analyse M1M3 Data

In [ ]:
fad = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    "*", 
    t_start,
    t_end)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    "*", 
    t_start,
    t_end)

fig, axs = plt.subplot_mosaic(
    mosaic="A",
    num="Slew Without Correction", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 4),
)

vandv.m1m3.timeline_zforces(axs["A"], fad, "zForce", elevation=el)

In [ ]:
fel = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedElevationForces", 
    "*", 
    t_start,
    t_end)

fba = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedBalanceForces", 
    "*",
    t_start,
    t_end)

fst = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedStaticForces",
    "*",
    t_start,
    t_end)

fao = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedActiveOpticForces",
    "*", 
    t_start,
    t_end)

series = [df.iloc[-1] for df in [fel, fba, fst, fad]]
labels = [
    "appliedElevationForces", 
    "appliedBalanceForces", 
    "appliedStaticForces",
    "forceActuatorData",
]

fig, axs = plt.subplot_mosaic(
    mosaic="AB\nAB\nCC\nCC",
    num="Slew Without Correction - Snapshot Forces", 
    constrained_layout=True,
    dpi=120,
    figsize=(9, 5),
)

vandv.m1m3.snapshot_xforces(axs["A"], series, labels=labels)
vandv.m1m3.snapshot_yforces(axs["B"], series, labels=labels)
vandv.m1m3.snapshot_zforces(axs["C"], series)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AB\nCD",
    num="Slew Without Correction - Snapshot zForces Overview", 
    constrained_layout=False,
    dpi=120,
    figsize=(10, 10),
)

vandv.m1m3.snapshot_zforces_overview(axs["A"], fad.iloc[-1], title="forceActuatorData")
vandv.m1m3.snapshot_zforces_overview(axs["B"], fel.iloc[-1], title="appliedElevationForces")
vandv.m1m3.snapshot_zforces_overview(axs["C"], fba.iloc[-1], title="appliedBalanceForces")
vandv.m1m3.snapshot_zforces_overview(axs["D"], fao.iloc[-1], title="appliedActiveOpticForces")

### Analyse M2

In [ ]:
axf = await client.select_time_series(
    "lsst.sal.MTM2.axialForce", 
    "*",
    t_start,
    t_end)

taf = await client.select_time_series(
    "lsst.sal.MTM2.tangentForce", 
    "*",
    t_start,
    t_end)

cof = await client.select_time_series(
    "lsst.sal.MTM2.command_applyForces",
    "*", 
    t_start,
    t_end)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="A\nB\nC",
    num="Slew Without Correction - M2", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 8),
)

_ = vandv.m2.timeline_axial_forces(axs["A"], axf, elevation=el)
_ = vandv.m2.timeline_axial_forces(axs["B"], axf, elevation=el, column="applied")
_ = vandv.m2.timeline_axial_forces(axs["C"], axf, elevation=el, column="lutGravity")

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="A\nB\nC\nD",
    num="Slew Without Correction - M2", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 8),
)

cols = [
    # "applied", 
    # "hardpointCorrection", 
    "lutGravity", 
    # "lutTemperature", 
    "measured"
]

_ = vandv.m2.timeline_axial_forces_per_act(axs["A"], axf, elevation=el, act="B1", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["B"], axf, elevation=el, act="B8", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["C"], axf, elevation=el, act="B16", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["D"], axf, elevation=el, act="B24", cols=cols)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="ABC",
    num="Slew Without Correction - M2 All Actuators", 
    constrained_layout=False,
    dpi=120,
    figsize=(12, 6),
)

_ = vandv.m2.snapshot_zforces_overview(axs["A"], axf.iloc[-1], ms=150, fs=6)
_ = vandv.m2.snapshot_zforces_overview(axs["B"], axf.iloc[-1], prefix="lutGravity", ms=150, fs=6)
_ = vandv.m2.snapshot_zforces_overview(axs["C"], axf.iloc[-1], prefix="applied", ms=150, fs=6)